#User Filters

In [188]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#to make the interactive maps
import folium
from folium.plugins import MarkerCluster, HeatMap

from ipywidgets import widgets, interact
from IPython.display import display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Load Data

In [ ]:
listings = pd.read_csv("/content/drive/MyDrive/SpatialProject/raw-data/SanFrancisco/listings.csv")

In [176]:
# Clean price column and convert it to a float64 type
listings['price'] = listings.price.str.replace('$', '')
listings['price'] = listings.price.str.replace(",", "")
listings['price'] = listings['price'].astype(str).astype(float)

#Ranking Algorithm

In [178]:
import math

maxNormalizationReviewMax = minMaxScalingReviewMin = sumNormalizationReviewSum = 0
count = 0

for index, row in listings.iterrows():
  if str(row["number_of_reviews"]).isnumeric():
    numReviews = str(row["number_of_reviews"])
  maxNormalizationReviewMax = maxNormalizationReviewMax if (maxNormalizationReviewMax > int(numReviews)) else int(numReviews)
  minMaxScalingReviewMin = minMaxScalingReviewMin if (minMaxScalingReviewMin < int(numReviews)) else int(numReviews)
  sumNormalizationReviewSum = sumNormalizationReviewSum + int(numReviews)
  count+=1

print(maxNormalizationReviewMax)
print(minMaxScalingReviewMin)
print(sumNormalizationReviewSum)
print(count)

960
0
331787
6629


In [179]:
count = 0
listings["rankMaxNormalizationScore"] = 0
listings["rankSumNormaizationScore"] = 0
listings["rankMinMaxScalinReviewCountScore"] = 0


#computing and assigning ranking score to each row
for index, row in listings.iterrows():
  if str(row["number_of_reviews"]).isnumeric():
 
    if row["price"] == 0:
      price= 1
    else:
      price = row["price"]
    weight = row["review_scores_value"] * (1 / price)
    maxNormalizationReviewRank = int(row["number_of_reviews"]) / maxNormalizationReviewMax
    sumNormalizationReviewRank = int(row["number_of_reviews"]) / sumNormalizationReviewSum
    minMaxScalingReviewRank = (int(row["number_of_reviews"]) - minMaxScalingReviewMin) / maxNormalizationReviewMax - minMaxScalingReviewMin

    finalMaxNormalizationScore = (math.log(maxNormalizationReviewRank) if (maxNormalizationReviewRank > 0) else 0)
    finalSumNormalizationScore = (math.log(sumNormalizationReviewRank) if (sumNormalizationReviewRank > 0) else 0)
    finalMinMaxScalingScore = (math.log(minMaxScalingReviewRank) if (minMaxScalingReviewRank > 0) else 0)

    row["rankMaxNormalizationScore"] = finalMaxNormalizationScore * weight
    row["rankSumNormaizationScore"] = finalSumNormalizationScore * weight
    row["rankMinMaxScalinReviewCountScore"] = finalMinMaxScalingScore * weight

    listings.loc[index] = row
    count += 1

  else:
    row["rankMaxNormalizationScore"] = 0
    row["rankSumNormaizationScore"] = 0
    row["rankMinMaxScalinReviewCountScore"] = 0 
    count += 1

print(count)

6629


In [181]:
listings.to_pickle("/content/drive/MyDrive/SpatialProject/raw-data/SanFrancisco/DF.pkl")

#User Filtering

In [183]:
room_type="Entire home/apt"
normalization_type = "Min-max Normalization"
max_price=999
min_reviews=0
min_nights=0
normalization_map = {
    "Max Normalization":"rankMaxNormalizationScore",
    "Sum Normalization" :"rankSumNormaizationScore",
    "Min-max Normalization" :"rankMinMaxScalinReviewCountScore"
  }

  
def f(room):
  global room_type
  room_type = room

def f1(price):
  global max_price
  max_price = price

def f2(nights):
  global min_nights
  min_nights = nights

def f3(rating):
  global min_reviews
  min_reviews = rating

def f4(normalization):
  global normalization_type
  normalization_type = normalization_map[normalization]
  
print("Choose a normalization type")
interact(f4, normalization=['Min-max Normalization','Sum Normalization','Max Normalization']);
print()
print("What room type are you looking for?")
interact(f, room=['Entire home/apt', 'Hotel room','Private room']);
print()
print("How many nights would you like to stay?")
interact(f2,  nights=widgets.IntSlider(min=1, max=30, step=1, value=0))
print()
print("Minimum Rating for the property?")
interact(f3,  rating=widgets.IntSlider(min=1, max=5, step=1, value=4))
print()
print("Maximum Price?")
interact(f1, price=widgets.IntSlider(min=0, max=1000, step=10, value=250))
print()

Choose a normalization type


interactive(children=(Dropdown(description='normalization', options=('Min-max Normalization', 'Sum Normalizati…


What room type are you looking for?


interactive(children=(Dropdown(description='room', options=('Entire home/apt', 'Hotel room', 'Private room'), …


How many nights would you like to stay?


interactive(children=(IntSlider(value=1, description='nights', max=30, min=1), Output()), _dom_classes=('widge…


Minimum Rating for the property?


interactive(children=(IntSlider(value=4, description='rating', max=5, min=1), Output()), _dom_classes=('widget…


Maximum Price?


interactive(children=(IntSlider(value=250, description='price', max=1000, step=10), Output()), _dom_classes=('…

In [184]:
room_type_df = listings.loc[listings['room_type'] == room_type]
price_df = listings.loc[listings['price'] < max_price]
nights_df = listings.loc[listings['minimum_nights'] > min_nights]
reviews_df = listings.loc[listings['review_scores_value'] >= min_reviews]

dfs = [room_type_df, price_df, nights_df, reviews_df]
merged_df= room_type_df.merge(price_df, how='inner').merge(nights_df, how='inner').merge(reviews_df,how='inner')
merged_df.sort_values(by=[normalization_type], ascending=False, inplace=True)

2012

#Rendering Maps

In [209]:
if(merged_df.empty == False):
  latitude = merged_df['latitude'].tolist()
  longi = merged_df['longitude'].tolist()
  locations = list(zip(latitude, longi))
  
  
  map1 = folium.Map(location=[latitude[0], longi[0]], zoom_start=11.5)
  marker_cluster = MarkerCluster().add_to(map1)

  count = 0
  for lat,lon,name, price, nights, review, listing_url, pic_url in zip(merged_df['latitude'], merged_df['longitude'], merged_df['name'], merged_df['price'], merged_df['minimum_nights'], merged_df['review_scores_value'], merged_df['listing_url'], merged_df['picture_url'] ):
    webname = '&nbsp;'.join(name.split(" "))
    if count < 10:
      marker = folium.Marker(location=[lat,lon], 
                           popup=f"<h5>{webname}</h5><br> <img src={pic_url} height=200, width=400> <b>Price:</b> ${int(price)}<br><b>Minimum Nights:</b> {nights}<br><b>Review:</b> {review}<br> <a href={listing_url}>Book on Airbnb!</a>",
                           icon=folium.Icon(color='green')
                           )  
      marker.add_to(marker_cluster)  
    else:
      marker = folium.Marker(location=[lat,lon], 
                           popup=f"<h5>{webname}</h5><br> <b>Price:</b> ${int(price)}<br><b>Minimum Nights:</b> {nights}<br><b>Review:</b> {review}<br> <a href={listing_url}>Book on Airbnb!</a>")
    
      marker.add_to(marker_cluster)  
    count += 1
  display(map1)

else:
   print('No Results Found')

In [210]:
top_10_df = merged_df.head(10)
if(merged_df.empty == False):
  latitude = top_10_df['latitude'].tolist()
  longi = top_10_df['longitude'].tolist()
  locations = list(zip(latitude, longi))
  
  
  map2 = folium.Map(location=[latitude[0], longi[0]], zoom_start=11.5)

  for lat,lon,name, price, nights, review, listing_url, pic_url in zip(top_10_df['latitude'], top_10_df['longitude'], top_10_df['name'], top_10_df['price'], top_10_df['minimum_nights'], top_10_df['review_scores_value'], top_10_df['listing_url'], top_10_df['picture_url'] ):
    webname = '&nbsp;'.join(name.split(" "))
    marker = folium.Marker(location=[lat,lon], 
                           popup=f"<h5>{webname}</h5><img src={pic_url} height=200, width=400> <br> <b>Price:</b> ${int(price)}<br><b>Minimum Nights:</b> {nights}<br><b>Review:</b> {review}<br> <a href={listing_url}>Book on Airbnb!</a>")
    marker.add_to(map2)  
  display(map2)

else:
   print('No Results Found')

In [211]:
top_100_df = merged_df.head(100)
if(merged_df.empty == False):
  latitude = top_100_df['latitude'].tolist()
  longi = top_100_df['longitude'].tolist()
  locations = list(zip(latitude, longi))
  
  
  map3 = folium.Map(location=[latitude[0], longi[0]], zoom_start=11.5)
  
  HeatMap(locations).add_to(map3)
  display(map3)

else:
   print('No Results Found')


#Flask 

In [198]:
!pip install flask-ngrok
!pip install flask==0.12.2



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.1 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [199]:
!pip install pyngrok==4.1.1
!ngrok authtoken 2HHToyYoGunNVoffuBPzEghQRzw_2YQqDTzVRzr22ZNzF9wBe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=cbfdc1309d8e57e68a0f26eb69c17e613bfcd4b334be16a17780acff84d3dfe1
  Stored in directory: /root/.cache/pip/wheels/5e/0a/51/8cb053ccd84481dd3233eba4cdb608bc7a885fd8ca418c0806
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [200]:
!mkdir templates
!touch templates/map.html

In [214]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta http-equiv="X-UA-Compatible" content="ie=edge">
    <title>Folium Map</title>
</head>
<body>
    <center>
    <h1>Airbnb</h1>
    <iframe class="map", src="/map1" width=90% height="600"></iframe>
    <h1>Top 10 Listings</h1>
    <iframe class="map", src="/map2" width=90% height="600"></iframe>
    <h1>Heatmap</h1>
    <iframe class="map", src="/map3" width=90% height="600"></iframe>
    </center>
</body>
</html>


Overwriting templates/index.html


In [217]:
# flask_ngrok_example.py
from flask import Flask, render_template
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route('/')
def index():
    map1.save('templates/map1.html')
    map2.save('templates/map2.html')
    map3.save('templates/map3.html')
    return render_template('index.html')

@app.route('/map1')
def map1_render():
    return render_template('map1.html')

@app.route('/map2')
def map2_render():
    return render_template('map2.html')

@app.route('/map3')
def map3_render():
    return render_template('map3.html')


if __name__ == '__main__':
    app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8395-104-198-62-173.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:00] "GET /map1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:00] "GET /map3 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:04] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:21] "GET /map1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:21] "GET /map2 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:21] "GET /map3 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Dec/2022 07:36:24] "GET /favicon.ico HTTP/1.1" 404 -
